In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [41]:
raw_data = np.loadtxt("inputs.txt")
raw_data_labels = np.loadtxt("labels.txt")
#print(raw_data.size)

#code to shuffle 2 arrays, and keep corresponding elements
randomize = np.arange(len(raw_data_labels)) 
np.random.shuffle(randomize) #creates a randomized sequence to be used as an index for the two arrays to shuffle them (https://www.delftstack.com/howto/numpy/python-numpy-shuffle-two-arrays/)

raw_data = raw_data[randomize]
raw_data_labels = raw_data_labels[randomize]

# print(raw_data)
# print(raw_data_labels)

#split into training, validation, testing
training_data = raw_data[:1200] 
training_data_labels = raw_data_labels[:1200] 

validation_data = raw_data[1200:1600] #note, includes start index, excludes end index
validation_data_labels = raw_data_labels[1200:1600]

testing_data = raw_data[1600:2000]
testing_data_labels = raw_data_labels[1600:2000]

print(validation_data_labels.shape)






(400,)
